In [1]:
using Jutul, JutulDarcy, HYPRE, GLMakie, MAT, Statistics

import Jutul: find_enclosing_cell, plot_mesh_edges, replace_variables!
import JutulDarcy: table_to_relperm, add_relperm_parameters!, brooks_corey_relperm
import JutulDarcy: KilloughHysteresis, ReservoirRelativePermeabilities

In [2]:
vars = matread("E:/temp/data_1272_64x64x8.mat")
perm_all = vars["perm"]
print("Perm all: ", size(perm_all))

Perm all: (1272, 64, 64, 8)

In [16]:
g = Jutul.gravity_constant
Darcy, bar, psi, kg, meter, hour, day, year = si_units(:darcy, :bar, :psi, :kilogram, :meter, :hour, :day, :year)

nx, ny, nz = 64, 64, 8
dx, dy, dz = 93.75, 93.75, 20.0

cart_dims     = (nx, ny, nz)
physical_dims = (nx*dx, ny*dy, nz*dz)
mesh          = UnstructuredMesh(CartesianMesh(cart_dims, physical_dims))
nc            = number_of_cells(mesh)

# points = mesh.node_points
# for (i, pt) in enumerate(points)
#     y, x, z = pt
#     x_u = 0.5 * π * x / 1000.0
#     w = 0.2
#     dz = 0.05*x + 0.05*abs(x - 2500.0)+ w*(30*cos(2.0*x_u) + 20*sin(5.0*x_u))
#     points[i] = pt + [0, 0, dz]
# end

32768

In [17]:
# setup rock
sample = 141

poro = fill(0.2, nc)
perm = zeros(3, nc)
kx = vec(perm_all[sample,:,:,:]) * Darcy ;
perm[1, :] = perm[2, :] = kx
perm[3, :] = 0.1 * kx

# setup fluid
so            = range(0, 1, 10)
sg            = range(0, 1, 50)
krog          = PhaseRelativePermeability(so, so.^2, label = :og)

tab_krg_drain = brooks_corey_relperm.(sg, n = 2, residual = 0.1)
tab_krg_imb   = brooks_corey_relperm.(sg, n = 3, residual = 0.25)

krg_drain     = PhaseRelativePermeability(sg, tab_krg_drain, label = :g)
krg_imb       = PhaseRelativePermeability(sg, tab_krg_imb, label = :g)

krg = (krg_drain, krg_imb)
H_g = KilloughHysteresis()
relperm = ReservoirRelativePermeabilities(g = krg, og = krog, hysteresis_g = H_g) ;

In [18]:
# setup reservoir
domain = reservoir_domain(mesh, permeability = perm, porosity = poro, temperature = convert_to_si(30.0, :Celsius))

# setup wells
w1, w2 = 0.25nx, 0.75nx
# Inj1 = setup_well(domain, [(w1, w1, i) for i in nz-1:nz], name = :Injector1, simple_well=true)
# Inj2 = setup_well(domain, [(w1, w2, i) for i in nz-1:nz], name = :Injector2, simple_well=true)
# Inj3 = setup_well(domain, [(w2, w2, i) for i in nz-1:nz], name = :Injector3, simple_well=true)
# Inj4 = setup_well(domain, [(w2, w1, i) for i in nz-1:nz], name = :Injector4, simple_well=true)
Inj1 = setup_well(domain, [(w1, w1, nz)], name = :Injector1, simple_well=true)
Inj2 = setup_well(domain, [(w1, w2, nz)], name = :Injector2, simple_well=true)
Inj3 = setup_well(domain, [(w2, w2, nz)], name = :Injector3, simple_well=true)
Inj4 = setup_well(domain, [(w2, w1, nz)], name = :Injector4, simple_well=true)

# setup model and update with relperms
model  = setup_reservoir_model(domain, :co2brine, wells = [Inj1,Inj2,Inj3,Inj4], extra_out = false, co2_physics = :kvalue)
replace_variables!(model, RelativePermeabilities = relperm)
add_relperm_parameters!(model) ;

In [19]:
plot_reservoir(model)

In [20]:
# initial state
p0 = zeros(nc)
depth = domain[:cell_centroids][3,:]
@. p0 = 200bar + depth * g * 1000.0

state0 = setup_reservoir_state(model, Pressure = p0, OverallMoleFractions = [1.0, 0.0])
parameters = setup_parameters(model) ;

In [21]:
# boundary conditions
boundary = Int[]
for cell in 1:nc
    I, J, K = cell_ijk(mesh, cell)
    if I == 1 || I == nx
        push!(boundary, cell)
    end
end
bc = flow_boundary_condition(boundary, domain, p0[boundary], fractional_flow = [1.0, 0.0])
println("Boundary condition added to $(length(bc)) cells.")

Boundary condition added to 1024 cells.


In [22]:
# setup schedule
#ramps = [1hour, 11hour, 12hour, (4day-24hour), (15day-4day)] #[1hour, 12hour, 24hour, 4day, 15day]
#nTrup = length(ramps)

Tinj   = 10year
dTinj  = year/12 / 2
nTinj  = Int(Tinj / dTinj)

Tmon   = 500year
dTmon  = 5year
nTmon  = Int(Tmon / dTmon)

dt_inj = repeat([dTinj], nTinj)
dt_mon = repeat([dTmon], nTmon)
#dt     = vcat(ramps, dt_inj, dt_mon)
dt     = vcat(dt_inj, dt_mon)

inj_rate    = 0.25 * 1e9 / 686.5266 / year
rate_target = TotalRateTarget(inj_rate)
I_ctrl      = InjectorControl(rate_target, [0.0, 1.0], density = 686.5266)

controls = Dict()
controls[:Injector1] = I_ctrl
controls[:Injector2] = I_ctrl
controls[:Injector3] = I_ctrl
controls[:Injector4] = I_ctrl

forces_inj = setup_reservoir_forces(model, control = controls, bc = bc)
forces_mon = setup_reservoir_forces(model, bc = bc)
#forces     = vcat(fill(forces_inj, nTrup), fill(forces_inj, nTinj), fill(forces_mon, nTmon))
forces     = vcat(fill(forces_inj, nTinj), fill(forces_mon, nTmon))
#println("$nTrup rampups + $nTinj report steps with injection, $nTmon report steps with migration.")
println("$nTinj report steps with injection, $nTmon report steps with migration.")

# run simulation
wd, states, t = simulate_reservoir(state0, model, dt, parameters = parameters, forces = forces,
                                   info_level = 1, 
                                   presolve_wells = false, #true
                                   linear_solver = :bicgstab, #:gmres
                                   precond = :cpr, #:ilu0
                                   max_nonlinear_iterations = 12,
                                   relaxation = true,
                                   tol_mb = 1e-3,
                                   rtol = 1e-3,
                                   max_timestep_cuts = 5)

240 report steps with injection, 100 report steps with migration.
Jutul: Simulating 510 years as 340 report steps
Step   1/340: Solving start to 2 weeks, 1.218 day, Δt = 2 weeks, 1.218 day 
Convergence: Report step 1, mini-step #5 (1 week, 17.24 hours) failed to converge. Reducing mini-step.
Step   2/340: Solving 2 weeks, 1.218 day to 4 weeks, 2.437 days, Δt = 2 weeks, 1.218 day 
Convergence: Report step 2, mini-step #1 (6 days, 22.72 hours) failed to converge. Reducing mini-step.
Convergence: Report step 2, mini-step #4 (5 days, 20.94 hours) failed to converge. Reducing mini-step.

┌ Warning: Exception occured in solve: ErrorException("Bad linear solve: Linear solver: maximum number of iterations exceeded, final residual: 0.23991192432715183, rel. value 2.9967447703224246. rtol = 0.001, atol = 1.0e-12, max_it = 50"). Attempting to cut time-step since failure_cuts_timestep = true.
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\simulator.jl:457
┌ Warning: Exception occurred in perform_step!
│   inner_exception = ErrorException("Bad linear solve: Linear solver: maximum number of iterations exceeded, final residual: 0.23991192432715183, rel. value 2.9967447703224246. rtol = 0.001, atol = 1.0e-12, max_it = 50")
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\simulator.jl:529



Step   3/340: Solving 4 weeks, 2.437 days to 6 weeks, 3.655 days, Δt = 2 weeks, 1.218 day 
Convergence: Report step 3, mini-step #2 (1 week, 3.814 days) failed to converge. Reducing mini-step.
Step   4/340: Solving 6 weeks, 3.655 days to 8 weeks, 4.874 days, Δt = 2 weeks, 1.218 day 
Step   5/340: Solving 8 weeks, 4.874 days to 10 weeks, 6.092 days, Δt = 2 weeks, 1.218 day 
Step   6/340: Solving 10 weeks, 6.092 days to 13 weeks, 7.455 hours, Δt = 2 weeks, 1.218 day 
Convergence: Report step 6, mini-step #1 (2 weeks, 1.218 day) failed to converge. Reducing mini-step.
Step   7/340: Solving 13 weeks, 7.455 hours to 15 weeks, 1.529 day, Δt = 2 weeks, 1.218 day 
Convergence: Report step 7, mini-step #1 (5 days, 16.97 hours) failed to converge. Reducing mini-step.
Step   8/340: Solving 15 weeks, 1.529 day to 17 weeks, 2.748 days, Δt = 2 weeks, 1.218 day 
Step   9/340: Solving 17 weeks, 2.748 days to 19 weeks, 3.966 days, Δt = 2 weeks, 1.218 day 
Convergence: Report step 9, mini-step #1 (2 we

┌ Warning: Exception occured in solve: ErrorException("Bad linear solve: Linear solver: maximum number of iterations exceeded, final residual: 0.4287578004196297, rel. value 2.38084457226834. rtol = 0.001, atol = 1.0e-12, max_it = 50"). Attempting to cut time-step since failure_cuts_timestep = true.
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\simulator.jl:457
┌ Warning: Exception occurred in perform_step!
│   inner_exception = ErrorException("Bad linear solve: Linear solver: maximum number of iterations exceeded, final residual: 0.4287578004196297, rel. value 2.38084457226834. rtol = 0.001, atol = 1.0e-12, max_it = 50")
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\simulator.jl:529



Step  11/340: Solving 21 weeks, 5.184 days to 23 weeks, 6.403 days, Δt = 2 weeks, 1.218 day 
Convergence: Report step 11, mini-step #1 (2 weeks, 1.218 day) failed to converge. Reducing mini-step.
Step  12/340: Solving 23 weeks, 6.403 days to 26 weeks, 14.91 hours, Δt = 2 weeks, 1.218 day 
Step  13/340: Solving 26 weeks, 14.91 hours to 28 weeks, 1.84 day, Δt = 2 weeks, 1.218 day 
Step  14/340: Solving 28 weeks, 1.84 day to 30 weeks, 3.058 days, Δt = 2 weeks, 1.218 day 
Convergence: Report step 14, mini-step #2 (1 week, 14.62 hours) failed to converge. Reducing mini-step.


...

Convergence: Report step 85, mini-step #4 (3 days, 19.31 hours) failed to converge. Reducing mini-step.


┌ Warning: Exception occured in solve: ErrorException("Bad linear solve: Linear solver: maximum number of iterations exceeded, final residual: 0.24686361567630544, rel. value 1.4639884772388223. rtol = 0.001, atol = 1.0e-12, max_it = 50"). Attempting to cut time-step since failure_cuts_timestep = true.
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\simulator.jl:457
┌ Warning: Exception occurred in perform_step!
│   inner_exception = ErrorException("Bad linear solve: Linear solver: maximum number of iterations exceeded, final residual: 0.24686361567630544, rel. value 1.4639884772388223. rtol = 0.001, atol = 1.0e-12, max_it = 50")
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\simulator.jl:529


Step  86/340: Solving 3 years, 28.26 weeks to 3 years, 30.44 weeks, Δt = 2 weeks, 1.218 day 
Step  87/340: Solving 3 years, 30.44 weeks to 3 years, 32.61 weeks, Δt = 2 weeks, 1.218 day 
Step  88/340: Solving 3 years, 32.61 weeks to 3 years, 34.78 weeks, Δt = 2 weeks, 1.218 day 
Step  89/340: Solving 3 years, 34.78 weeks to 3 years, 36.96 weeks, Δt = 2 weeks, 1.218 day 
Step  90/340: Solving 3 years, 36.96 weeks to 3 years, 39.13 weeks, Δt = 2 weeks, 1.218 day 
Step  91/340: Solving 3 years, 39.13 weeks to 3 years, 41.31 weeks, Δt = 2 weeks, 1.218 day 
Step  92/340: Solving 3 years, 41.31 weeks to 3 years, 43.48 weeks, Δt = 2 weeks, 1.218 day 
Step  93/340: Solving 3 years, 43.48 weeks to 3 years, 45.66 weeks, Δt = 2 weeks, 1.218 day 
Step  94/340: Solving 3 years, 45.66 weeks to 3 years, 47.83 weeks, Δt = 2 weeks, 1.218 day 
Step  95/340: Solving 3 years, 47.83 weeks to 3 years, 50 weeks, Δt = 2 weeks, 1.218 day 
Step  96/340: Solving 3 years, 50 weeks to 4 years, Δt = 2 weeks, 1.218 d

┌ Error: Simulation did not complete successfully.
└ @ Jutul C:\Users\mmorales\.julia\packages\Jutul\awbiL\src\simulator\print.jl:179


ReservoirSimResult with 240 entries:

  wells (4 present):
    :Injector3
    :Injector1
    :Injector2
    :Injector4
    Results per well:
       :H2O_mass_rate => Vector{Float64} of size (240,)
       :lrat => Vector{Float64} of size (240,)
       :orat => Vector{Float64} of size (240,)
       :control => Vector{Symbol} of size (240,)
       :bhp => Vector{Float64} of size (240,)
       :CO2_mass_rate => Vector{Float64} of size (240,)
       :mass_rate => Vector{Float64} of size (240,)
       :rate => Vector{Float64} of size (240,)
       :grat => Vector{Float64} of size (240,)

  states (Vector with 240 entries, reservoir variables for each state)
    :LiquidMassFractions => Matrix{Float64} of size (2, 32768)
    :PhaseMassDensities => Matrix{Float64} of size (2, 32768)
    :OverallMoleFractions => Matrix{Float64} of size (2, 32768)
    :Saturations => Matrix{Float64} of size (2, 32768)
    :Pressure => Vector{Float64} of size (32768,)
    :VaporMassFractions => Matrix{Float64} of 

In [23]:
plot_reservoir(model, states)

In [ ]:
inventory = co2_inventory(model, wd, states, t)
JutulDarcy.plot_co2_inventory(t, inventory)

***
# Functional